In [8]:
import re
import jieba
import gensim
from gensim import corpora, models, similarities
from gensim.models import KeyedVectors
from pprint import pprint
from collections import defaultdict

In [ ]:
def load_stopwords(stopwords_path):
    with open(stopwords_path, 'r', encoding='utf-8') as f:
        return [line.strip() for line in f]
    
def preprocess_data(corpus_path, stopwords):
    corpus = []
    with open(corpus_path, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            line = re.sub(r"[0-9\s+\.\!\/_,$%^*()?;；:-【】+\"\']+|[+——！，;:。？、~@#￥%……&*（）]+", " ", line)
            corpus.append(' '.join([word for word in jieba.lcut(line) if word != " " and word != "\t" and word not in stopwords]))
    return corpus

def cut_low_freq_word(documents, thre):
    texts = [[word for word in document.split()] for document in documents]
    frequency = defaultdict(int)
    for text in texts:
        for token in text:
            frequency[token] += 1
    return [(" ").join([token for token in text if frequency[token] > thre]) for text in texts]

In [2]:
stopwords_path = "./data/stop_words.txt"
doc_title_east = "./data/title_east.txt"
doc_title_apple= "./data/title_apple.txt"

In [3]:
stopwords = load_stopwords(stopwords_path)
corp_title_east = preprocess_data(doc_title_east, stopwords)
corp_title_apple = preprocess_data(doc_title_apple, stopwords)

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/m3/4yh806w92fdgcn0bk16ql7nw0000gn/T/jieba.cache
Loading model cost 0.625 seconds.
Prefix dict has been built successfully.


In [4]:
texts_title_east = cut_low_freq_word(corp_title_east, 3)
texts_title_apple = cut_low_freq_word(corp_title_apple, 3)

In [30]:
model_w2v = gensim.models.Word2Vec(texts_title_east, size=300)
model_w2v.intersect_word2vec_format('./data/sgns.financial.char.bz2',
                                lockf=1.0,
                                encoding='utf8',
                                )
model_w2v.train(texts_title_east,total_examples=model_w2v.corpus_count, epochs=10)

KeyboardInterrupt: 

In [18]:
wv = KeyedVectors.load_word2vec_format('./data/sgns.financial.char.bz2')